In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import numpy as np

#from tabulate import tabulate

In [ ]:
from datetime import datetime

In [ ]:
#!pip install pymysql

In [ ]:
#!Pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from PIL import Image
import datetime as DT
import time

In [ ]:
from re import search
import csv
import os
import requests
from sqlalchemy import create_engine
import json
import pymysql

In [ ]:
# Credentials to database connection
hostname="mysql_db_ip_address"
dbname="db_name"
uname="db_user_name"
pwd="db_password"
xchTxTable="xchtrans"
# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, db=dbname, user=uname, pw=pwd))

In [ ]:
xchWallet='xch wallet'


In [ ]:
csv_temp_file='temp file.csv'
csv_final_file='final file.csv'

In [ ]:
def insertToDB(df_,insert_mode='append'):
    # Convert dataframe to sql table                                   
    df_.to_sql(xchTxTable, engine,if_exists=insert_mode, index=False)

In [ ]:
def getTrans(offset_=0,limit_=25,wallet_=xchWallet):
    url_='https://xchscan.com/api/account/txns?address='+ wallet_ +'&limit='+ str(limit_) +'&offset=' + str(offset_)
    req = requests.get(url_)
    return req.json()['txns']


In [ ]:
def getAllTrans(wallet_):
    o=0
    l=25
    next = True
    data_=[]
    while next:
        txs = getTrans(o,l, wallet_)
        #print (len(txs))
        if(len(txs)> 0):
            data_.extend(txs)
        if(len(txs) < l):
            next = False
        else:            
            o = o + l
    return data_            

In [ ]:
def getTransaction(wallet_,all_=False):
    if all_:
        return getAllTrans(wallet_)
    else :
        return getTrans(0,25,wallet_)

In [ ]:
def getTransactionFromXchscan(wallet_,all_=False):
    data_ = getTransaction(wallet_,all_)
    df = pd.DataFrame(data_)
    df_formatted = df[['timestamp','from','to','amount']]
    
    return df_formatted

In [ ]:
def getLastTransactionFromDB():
    df2 = pd.read_sql('SELECT * FROM ' + xchTxTable + ' where timestamp = (select max(timestamp) from ' + xchTxTable + ')', con=engine)
    return df2.timestamp[0]

In [ ]:

def writeCsvFile(fname,data_):           
    outFile=fname        
    with open(outFile, 'w',newline='' ) as csvfile:
        csvwriter = csv.writer(csvfile)        
        for line in data_:  
            csvwriter.writerow(line)

In [ ]:
def getNewRow(df_):
    lastPayout = getLastTransactionFromDB()
    print(lastPayout)
    r_index = 0
    for index, row in df_.iterrows():
        if (row.timestamp == lastPayout):
            print(index)
            r_index = index
            break
        else :
            r_index = index
            print (row.timestamp+ ' '+ lastPayout)
    print('new Row' + str(r_index))    
    return df_[:r_index]      

In [ ]:
def getTransactionsToDb(first_time=False):
    dfx =  getTransactionFromXchscan(xchWallet,first_time)
    if(first_time == True):
        insertToDB(dfx,'replace')
    else:
        try:            
            newRow=getNewRow(dfx)            
            insertToDB(newRow,'append')
        except Exception as e:
            print('No new Transaction')
            print (e)

In [ ]:
### folowing function for getTransaction from Database and generate to CSV file for create report

In [ ]:

def createTempCsv():
    df2 = pd.read_sql('SELECT * FROM ' + xchTxTable , con=engine)
    df2['xch']=df2['amount']/1000000000000
    df2['date_time'] =  pd.to_datetime(df2['timestamp'], unit='s',  errors='coerce')
    #https://towardsdatascience.com/10-tricks-for-converting-numbers-and-strings-to-datetime-in-pandas-82a4645fc23d

    df2['month'] = df2['date_time'].dt.strftime('%Y-%m')
    p = df2.pivot_table(['xch'], index='month', aggfunc=['count','sum'],margins=True)
    p.columns=['Transfer_time','Amt_xch']
    p = p.sort_values(by='month', ascending = False)
    p['Amt2'] = p['Amt_xch'].astype(str)
    p.reset_index(inplace = False)
    p.T.drop('Amt2').to_csv(csv_temp_file,index=True,float_format='%.5f') 

In [ ]:
def createCsvForReport():
    with open(csv_temp_file) as file:
        content = file.readlines()
        header = content[:1]
        rows = content[1:]
        rows=content
        new_content=[] 
        #new_content.append(header)

        for i in range(len(rows)):
            r =rows[i]
            cls = r.split(',')
            if i == 2:
                res = [str("{:.5f}".format(float(cls[value]))) for value in range(1,len(cls))]
                res.insert(0,cls[0])
                new_content.append(res)
            else:
                new_content.append(cls)

        writeCsvFile(csv_final_file,new_content) 

In [ ]:
# update Transaction to local database
getTransactionsToDb(True)
# create csv file for report
createTempCsv()
createCsvForReport()

In [ ]:
#!jupyter nbconvert --to script XCHTranMysql.ipynb